In [468]:
import os, re
import chardet
import subprocess
import unicodecsv as csv
import numpy as np
import pandas as pd 
import re

In [469]:
obj = subprocess.Popen(["cmd"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, encoding="utf-8")
obj.stdin.write("abaqus licensing dslsstat -usage")
obj.stdin.write("\n")
obj.stdin.close()

In [470]:
cmd_out = obj.stdout.read()
obj.stdout.close()
cmd_error = obj.stderr.read()
obj.stderr.close()

In [471]:
cmd_out2 = cmd_out.splitlines()
#cmd_out2 = cmd_out2.split('|')

In [472]:
out3 = np.array(cmd_out2).T

In [473]:
df=pd.DataFrame(out3)
df=df[0].str.split('|', expand=True)
df1=df[0].str.split(',', expand=True)
df1[['a','b','c','day','time','f','linum','h']]=df1[1].str.split('\s', expand=True)
#df=pd.concat([df1,df2], axis=1)
df=df1.drop(df1.tail(2).index) #从尾部去掉 n 行
df.to_csv('result.csv')

In [ ]:
print(df)
print(cmd_error)  # 程序没有异常，只输出空行

                                                     0  \
0           Microsoft Windows [Version 10.0.22000.434]   
1      (c) Microsoft Corporation. All rights reserved.   
2                                                        
3    (base) C:\Users\brcy\Desktop>abaqus licensing ...   
4                       # 1 license servers declared:    
..                                                 ...   
299      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...   
300      bridge on BRABQ9 (42CE189EF33B866F-0a6c330...   
301      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...   
302      bridge on BRABQ10 (440E1CECEF40E5FD-0a6c33...   
303      bridge on BRABQ8 (41E614006A846887-0a6c33c...   

                                                    1     a        b     c  \
0                                                None  None     None  None   
1                                                None  None     None  None   
2                                                None  None     None 

In [474]:
for idx in reversed(df.index):
    if df.loc[idx,'linum'] != None :
        print(idx, df.loc[idx, 0], df.loc[idx,'day'], df.loc[idx,'time'], df.loc[idx, 'linum'])
    else: 
        break

301     bridge on BRABQ8 (41E614006A846887-0a6c3362.2) 21-1-2022 15:40:32 76
300     bridge on BRABQ10 (440E1CECEF40E5FD-0a6c330a.2) 21-1-2022 14:47:47 32
299     hiratani on DESKTOP-FCQDINT (42E51085C2CF9139-0a23e27d.2) 21-1-2022 13:22:25 50
298     bridge on BRABQ9 (42CE189EF33B866F-0a6c330c.1) 21-1-2022 11:14:10 76
297     hiratani on DESKTOP-FCQDINT (42E51085C2CF9139-0a23e27d.2) 20-1-2022 16:48:40 50
296     hiratani on DESKTOP-FCQDINT (42E51085C2CF9139-0a23e27d.2) 20-1-2022 16:09:15 24
295     hiratani on DESKTOP-FCQDINT (42E51085C2CF9139-0a23e27d.2) 20-1-2022 15:23:58 50
294     bridge on BRABQ10 (440E1CECEF40E5FD-0a6c330a.2) 21-1-2022 14:47:47 40
293     bridge on BRABQ10 (440E1CECEF40E5FD-0a6c330a.2) 21-1-2022 14:47:47 4
292     hiratani on DESKTOP-FCQDINT (42E51085C2CF9139-0a23e27d.2) 20-1-2022 16:18:51 40
291     hiratani on DESKTOP-FCQDINT (42E51085C2CF9139-0a23e27d.2) 20-1-2022 16:09:15 26
290     hiratani on DESKTOP-FCQDINT (42E51085C2CF9139-0a23e27d.2) 20-1-2022 16:18:51 

In [481]:
df5=df.drop(df.head(idx).index)
df5=df5.drop([1,'a','b','c'], axis=1)
print(df5)

                                                     0        day      time  \
289                                                          None      None   
290      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...  20-1-2022  16:18:51   
291      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...  20-1-2022  16:09:15   
292      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...  20-1-2022  16:18:51   
293      bridge on BRABQ10 (440E1CECEF40E5FD-0a6c33...  21-1-2022  14:47:47   
294      bridge on BRABQ10 (440E1CECEF40E5FD-0a6c33...  21-1-2022  14:47:47   
295      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...  20-1-2022  15:23:58   
296      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...  20-1-2022  16:09:15   
297      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...  20-1-2022  16:48:40   
298      bridge on BRABQ9 (42CE189EF33B866F-0a6c330...  21-1-2022  11:14:10   
299      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...  21-1-2022  13:22:25   
300      bridge on BRABQ10 (440E1CECEF40E5FD-0a6c33.

In [493]:
df5['linum'] = df5['linum'].astype(float)
df6=df5.groupby([0,'time'], sort=False)['linum'].sum().reset_index()
lisum=df6['linum'].sum()
df6.loc[0]=['sum','570-',lisum]
print(df6)

                                                   0      time  linum
0                                                sum      570-  478.0
1      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...  16:09:15   50.0
2      bridge on BRABQ10 (440E1CECEF40E5FD-0a6c33...  14:47:47   76.0
3      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...  15:23:58   50.0
4      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...  16:48:40   50.0
5      bridge on BRABQ9 (42CE189EF33B866F-0a6c330...  11:14:10   76.0
6      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...  13:22:25   50.0
7      bridge on BRABQ8 (41E614006A846887-0a6c336...  15:40:32   76.0


In [29]:
#*-* encoding: utf-8 *-*
import os, re
import chardet
import subprocess
import unicodecsv as csv
import numpy as np
import pandas as pd 
import re
from pandas.plotting import  table
import matplotlib.pyplot as plt
import dataframe_image as dfi
import base64
import json
import requests
from base64 import b64encode

obj = subprocess.Popen(["cmd"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, encoding="utf-8")
obj.stdin.write("abaqus licensing dslsstat -usage")
obj.stdin.write("\n")
obj.stdin.close()

cmd_out = obj.stdout.read()
obj.stdout.close()
cmd_error = obj.stderr.read()
obj.stderr.close()

cmd_out2 = cmd_out.splitlines()
#cmd_out2 = cmd_out2.split('|')

out3 = np.array(cmd_out2).T

df=pd.DataFrame(out3)
df=df[0].str.split('|', expand=True)
df1=df[0].str.split(',', expand=True)
df1[['a','b','c','day','time','f','linum','h']]=df1[1].str.split('\s', expand=True)
#df=pd.concat([df1,df2], axis=1)
df=df1.drop(df1.tail(2).index) #从尾部去掉 n 行

for idx in reversed(df.index):
    if df.loc[idx,'linum'] == None :
        break
df5=df.drop(df.head(idx).index)
df5=df5.drop([1,'a','b','c'], axis=1)
df5['linum'] = df5['linum'].astype(float)
df6=df5.groupby([0,'time'], sort=False)['linum'].sum().reset_index()
lisum=df6['linum'].sum()
remin='remaing:'+str(570-lisum)
df6.loc[0]=['All lisence is 570',remin,lisum]
print(df6)
#########################
#グラフ化
#########################
# fig = plt.figure(figsize=(3, 4), dpi=1400)
# ax = fig.add_subplot(111, frame_on=False) 
# ax.xaxis.set_visible(False)  # hide the x axis
# ax.yaxis.set_visible(False)  # hide the y axis
# table(ax, df6, loc='center')  # 将df换成需要保存的dataframe即可
#plt.savefig('license.png')
df_styled = df6.style.background_gradient()
dfi.export(df_styled,"V:\wiki\public\images\lisence.png")
#df6.to_csv('result.csv')

                                                   0          time  linum
0                                 All lisence is 570  remaing:72.0  498.0
1      bridge on BRABQ10 (440E1CECEF40E5FD-0a6c33...      14:47:47   76.0
2      bridge on BRABQ8 (41E614006A846887-0a6c33c...      18:45:14   76.0
3      bridge on BRABQ9 (42CE189EF33B866F-0a6c330...      11:14:10   76.0
4      hiratani on DESKTOP-FCQDINT (42E51085C2CF9...      13:22:25   50.0
5      bridge on BRABQ10 (440E1CECEF40E5FD-0a6c33...      17:25:32  120.0
6      azuma on DESKTOP-LR7USU2 (424C1C52825ABB52...      18:50:55   50.0


In [28]:
pyinstaller -F lisence_test.py --clean

SyntaxError: invalid syntax (Temp/ipykernel_24656/3101482062.py, line 1)